## 00__preprocess

in this notebook, i take the output from feature counts (counts per gene in HUES64 and mESCs) and prepare it for DESeq2 analysis.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import itertools
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import sys

from scipy.stats import spearmanr

# import utils
sys.path.append("../../../utils")
from plotting_utils import *

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
mpl.rcParams['figure.autolayout'] = False

In [2]:
sns.set(**PAPER_PRESET)
fontsize = PAPER_FONTSIZE

In [3]:
np.random.seed(2019)

## variables

In [4]:
## note: should probably re-run hESC samples w/o lifted gencode annotations...

In [5]:
rna_seq_dir = "../../../data/03__rna_seq/02__for_DESeq2"

In [6]:
## CHANGE THESE WHEN FILES MOVE
hESC_rep1_f = "/n/rinn_data2/users/kaia/mpra/all_evo_mpra/2019__TSS_evolution_MPRA/misc/06__rna_seq/00__HUES64/04__featurecounts/hESC_rep1.counts.txt"
hESC_rep2_f = "/n/rinn_data2/users/kaia/mpra/all_evo_mpra/2019__TSS_evolution_MPRA/misc/06__rna_seq/00__HUES64/04__featurecounts/hESC_rep2.counts.txt"
mESC_rep1_f = "/n/rinn_data2/users/kaia/mpra/all_evo_mpra/2019__TSS_evolution_MPRA/misc/06__rna_seq/01__mESC/04__featurecounts/mESC_rep1.counts.txt"
mESC_rep2_f = "/n/rinn_data2/users/kaia/mpra/all_evo_mpra/2019__TSS_evolution_MPRA/misc/06__rna_seq/01__mESC/04__featurecounts/mESC_rep2.counts.txt"
mESC_rep3_f = "/n/rinn_data2/users/kaia/mpra/all_evo_mpra/2019__TSS_evolution_MPRA/misc/06__rna_seq/01__mESC/04__featurecounts/mESC_rep3.counts.txt"

In [7]:
orth_f = "../../../misc/01__ensembl_orthologs/ensembl96_human_mouse_orths.txt.gz"

## 1. import data

In [8]:
hESC_rep1 = pd.read_table(hESC_rep1_f, skiprows=1)
hESC_rep2 = pd.read_table(hESC_rep2_f, skiprows=1)
hESC_rep1.head()

Geneid                                                Chr  \
0  ENSG00000223972.5_1                                chr1;chr1;chr1;chr1   
1  ENSG00000227232.5_1  chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...   
2  ENSG00000243485.4_2                                     chr1;chr1;chr1   
3  ENSG00000237613.2_1                                     chr1;chr1;chr1   
4  ENSG00000268020.3_1                                               chr1   

                                               Start  \
0                            11869;12613;12975;13221   
1  14404;15005;15796;16607;16858;17233;17606;1791...   
2                                  29554;30267;30976   
3                                  34554;35245;35721   
4                                              52473   

                                                 End                 Strand  \
0                            12227;12721;13052;14409                +;+;+;+   
1  14501;15038;15947;16765;17055;17368;17742;1806...  -;-;-;-;-;-;-;-;-;-;-   
2                                  30039;30667;31109                  +;+;+   
3                                  35174;35481;36081                  -;-;-   
4                                              53312                      +   

   Length  ../00__HUES64/03__alignments/hESC_rep1/accepted_hits.bam  
0    1735                                                  2         
1    1351                                                 45         
2    1021                                                  3         
3    1219                                                  0         
4     840                                                  0

In [9]:
mESC_rep1 = pd.read_table(mESC_rep1_f, skiprows=1)
mESC_rep2 = pd.read_table(mESC_rep2_f, skiprows=1)
mESC_rep3 = pd.read_table(mESC_rep3_f, skiprows=1)
mESC_rep1.head()

Geneid                  Chr                            Start  \
0  ENSMUSG00000102693.1                 chr1                          3073253   
1  ENSMUSG00000064842.1                 chr1                          3102016   
2  ENSMUSG00000051951.5  chr1;chr1;chr1;chr1  3205901;3213439;3421702;3670552   
3  ENSMUSG00000102851.1                 chr1                          3252757   
4  ENSMUSG00000103377.1                 chr1                          3365731   

                               End   Strand  Length  \
0                          3074322        +    1070   
1                          3102125        +     110   
2  3207317;3216968;3421901;3671498  -;-;-;-    6094   
3                          3253236        +     480   
4                          3368549        -    2819   

   ../01__mESC/03__alignments/mESC_rep1/accepted_hits.bam  
0                                                  0       
1                                                  0       
2                                                  2       
3                                                  0       
4                                                  0

In [10]:
orth = pd.read_table(orth_f)
orth.head()

Gene stable ID Transcript stable ID Gene name Mouse gene stable ID  \
0  ENSG00000198888      ENST00000361390    MT-ND1   ENSMUSG00000064341   
1  ENSG00000198763      ENST00000361453    MT-ND2   ENSMUSG00000064345   
2  ENSG00000198804      ENST00000361624    MT-CO1   ENSMUSG00000064351   
3  ENSG00000198712      ENST00000361739    MT-CO2   ENSMUSG00000064354   
4  ENSG00000228253      ENST00000361851   MT-ATP8   ENSMUSG00000064356   

  Mouse protein or transcript stable ID Mouse gene name  \
0                    ENSMUSP00000080991          mt-Nd1   
1                    ENSMUSP00000080992          mt-Nd2   
2                    ENSMUSP00000080993          mt-Co1   
3                    ENSMUSP00000080994          mt-Co2   
4                    ENSMUSP00000080995         mt-Atp8   

   %id. target Mouse gene identical to query gene  \
0                                         77.0440   
1                                         57.0605   
2                                         90.8382   
3                                         71.3656   
4                                         45.5882   

   %id. query gene identical to target Mouse gene  \
0                                         77.0440   
1                                         57.3913   
2                                         90.6615   
3                                         71.3656   
4                                         46.2687   

   Mouse orthology confidence [0 low, 1 high] Mouse homology type  
0                                         1.0    ortholog_one2one  
1                                         1.0    ortholog_one2one  
2                                         1.0    ortholog_one2one  
3                                         1.0    ortholog_one2one  
4                                         0.0    ortholog_one2one

## 2. join counts files

In [11]:
hESC = hESC_rep1[["Geneid", 
                  "../00__HUES64/03__alignments/hESC_rep1/accepted_hits.bam"]].merge(hESC_rep2[["Geneid",
                                                                                                "../00__HUES64/03__alignments/hESC_rep2/accepted_hits.bam"]],
                                                                                     on="Geneid")
hESC.columns = ["long_gene_id", "rep1", "rep2"]
hESC["gene_id"] = hESC["long_gene_id"].str.split(".", expand=True)[0]
hESC.sample(5)

long_gene_id  rep1  rep2          gene_id
21266   ENSG00000205186.2_1     0     0  ENSG00000205186
41539   ENSG00000263393.1_1    15     7  ENSG00000263393
26802  ENSG00000175220.11_2  3707  2346  ENSG00000175220
6469    ENSG00000233729.1_1     2     5  ENSG00000233729
28811   ENSG00000177406.4_1    93   113  ENSG00000177406

In [12]:
mESC = mESC_rep1[["Geneid", 
                  "../01__mESC/03__alignments/mESC_rep1/accepted_hits.bam"]].merge(mESC_rep2[["Geneid",
                                                                                              "../01__mESC/03__alignments/mESC_rep2/accepted_hits.bam"]],
                                                                                   on="Geneid").merge(mESC_rep3[["Geneid",
                                                                                                                 "../01__mESC/03__alignments/mESC_rep3/accepted_hits.bam"]],
                                                                                                      on="Geneid")
mESC.columns = ["long_gene_id", "rep1", "rep2", "rep3"]
mESC["gene_id"] = mESC["long_gene_id"].str.split(".", expand=True)[0]
mESC.sample(5)

long_gene_id  rep1  rep2  rep3             gene_id
47873   ENSMUSG00000015665.8     0     0     0  ENSMUSG00000015665
15110   ENSMUSG00000054675.5     8     4     1  ENSMUSG00000054675
4765   ENSMUSG00000049044.16    42    54    87  ENSMUSG00000049044
11958   ENSMUSG00000054958.6     2     2     2  ENSMUSG00000054958
13921   ENSMUSG00000106292.1     0     0     0  ENSMUSG00000106292

## 3. do some very quick QC

In [13]:
orth["Mouse homology type"].value_counts()

ortholog_one2one      132285
ortholog_one2many      16840
ortholog_many2many     13324
Name: Mouse homology type, dtype: int64

In [14]:
orth[orth["Gene name"].isin(["POU5F1", "NANOG", 
                             "SOX2", "XIST", "EOMES"])][["Gene stable ID", "Gene name", 
                                                         "Mouse gene stable ID", "Mouse gene name"]].drop_duplicates()

Gene stable ID Gene name Mouse gene stable ID Mouse gene name
18291   ENSG00000181449      SOX2   ENSMUSG00000074637            Sox2
23293   ENSG00000204531    POU5F1   ENSMUSG00000024406          Pou5f1
31340   ENSG00000163508     EOMES   ENSMUSG00000032446           Eomes
107084  ENSG00000111704     NANOG   ENSMUSG00000012396           Nanog

In [15]:
hESC[hESC["gene_id"].isin(["ENSG00000181449", "ENSG00000204531", "ENSG00000163508", "ENSG00000111704"])]

long_gene_id   rep1   rep2          gene_id
8147   ENSG00000163508.12_1   2372   1949  ENSG00000163508
10092   ENSG00000181449.3_2  17777  13935  ENSG00000181449
15741  ENSG00000204531.16_2  54643  33314  ENSG00000204531
29004  ENSG00000111704.10_1   6327   4139  ENSG00000111704

In [16]:
mESC[mESC["gene_id"].isin(["ENSMUSG00000074637", "ENSMUSG00000024406", "ENSMUSG00000032446", "ENSMUSG00000012396"])]

long_gene_id   rep1   rep2   rep3             gene_id
7612    ENSMUSG00000074637.7  23304  24661  30664  ENSMUSG00000074637
18622  ENSMUSG00000012396.12  41275  42557  53366  ENSMUSG00000012396
29266  ENSMUSG00000032446.14    496    650    814  ENSMUSG00000032446
43569  ENSMUSG00000024406.16  42454  45654  58705  ENSMUSG00000024406

In [17]:
xist_human = "ENSG00000229807"
sry_human = "ENSG00000184895"

In [18]:
xist_mouse = "ENSMUSG00000086503"
sry_mouse = "ENSMUSG00000069036"

In [19]:
hESC[hESC["gene_id"].isin([xist_human, sry_human])]

long_gene_id  rep1  rep2          gene_id
48761  ENSG00000229807.10_2     8     4  ENSG00000229807
49843   ENSG00000184895.7_1    74    53  ENSG00000184895

In [20]:
mESC[mESC["gene_id"].isin([xist_mouse, sry_mouse])]

long_gene_id  rep1  rep2  rep3             gene_id
47979  ENSMUSG00000086503.3    32    24    46  ENSMUSG00000086503
49034  ENSMUSG00000069036.3     0     0     0  ENSMUSG00000069036

## 4. prepare counts for DESeq2 library estimation (all genes)

In [25]:
hESC = hESC[["gene_id", "rep1", "rep2"]].drop_duplicates(subset="gene_id")
print(len(hESC))
print(len(hESC.gene_id.unique()))
hESC.head()

60211
60211


gene_id  rep1  rep2
0  ENSG00000223972     2     0
1  ENSG00000227232    45    36
2  ENSG00000243485     3     0
3  ENSG00000237613     0     0
4  ENSG00000268020     0     0

In [22]:
mESC = mESC[["gene_id", "rep1", "rep2", "rep3"]].drop_duplicates()
print(len(mESC))
print(len(mESC.gene_id.unique()))
mESC.head()

50600
50600


gene_id  rep1  rep2  rep3
0  ENSMUSG00000102693     0     0     0
1  ENSMUSG00000064842     0     0     0
2  ENSMUSG00000051951     2     2     2
3  ENSMUSG00000102851     0     0     0
4  ENSMUSG00000103377     0     0     0

In [23]:
hESC_cols = {"rep1": ["rep1"], "rep2": ["rep2"]}
hESC_cols = pd.DataFrame.from_dict(hESC_cols, orient="index").reset_index()
hESC_cols.columns = ["column", "condition"]
hESC_cols.head()

column condition
0   rep1      rep1
1   rep2      rep2

In [24]:
mESC_cols = {"rep1": ["rep1"], "rep2": ["rep2"], "rep3": ["rep3"]}
mESC_cols = pd.DataFrame.from_dict(mESC_cols, orient="index").reset_index()
mESC_cols.columns = ["column", "condition"]
mESC_cols.head()

column condition
0   rep1      rep1
1   rep2      rep2
2   rep3      rep3

## 5. prepare counts for DESeq2 differential expression analysis

In [26]:
orth["Mouse homology type"].value_counts()

ortholog_one2one      132285
ortholog_one2many      16840
ortholog_many2many     13324
Name: Mouse homology type, dtype: int64

In [27]:
# remove many2many orths
orth_sub = orth[orth["Mouse homology type"] != "ortholog_many2many"]
orth_sub.head()

Gene stable ID Transcript stable ID Gene name Mouse gene stable ID  \
0  ENSG00000198888      ENST00000361390    MT-ND1   ENSMUSG00000064341   
1  ENSG00000198763      ENST00000361453    MT-ND2   ENSMUSG00000064345   
2  ENSG00000198804      ENST00000361624    MT-CO1   ENSMUSG00000064351   
3  ENSG00000198712      ENST00000361739    MT-CO2   ENSMUSG00000064354   
4  ENSG00000228253      ENST00000361851   MT-ATP8   ENSMUSG00000064356   

  Mouse protein or transcript stable ID Mouse gene name  \
0                    ENSMUSP00000080991          mt-Nd1   
1                    ENSMUSP00000080992          mt-Nd2   
2                    ENSMUSP00000080993          mt-Co1   
3                    ENSMUSP00000080994          mt-Co2   
4                    ENSMUSP00000080995         mt-Atp8   

   %id. target Mouse gene identical to query gene  \
0                                         77.0440   
1                                         57.0605   
2                                         90.8382   
3                                         71.3656   
4                                         45.5882   

   %id. query gene identical to target Mouse gene  \
0                                         77.0440   
1                                         57.3913   
2                                         90.6615   
3                                         71.3656   
4                                         46.2687   

   Mouse orthology confidence [0 low, 1 high] Mouse homology type  
0                                         1.0    ortholog_one2one  
1                                         1.0    ortholog_one2one  
2                                         1.0    ortholog_one2one  
3                                         1.0    ortholog_one2one  
4                                         0.0    ortholog_one2one

In [28]:
# subset to genes only
orth_genes = orth_sub[["Gene stable ID", "Gene name", "Mouse gene stable ID", "Mouse gene name"]].drop_duplicates()
orth_genes.columns = ["gene_id_human", "gene_name_human", "gene_id_mouse", "gene_name_mouse"]
orth_genes.head()

gene_id_human gene_name_human       gene_id_mouse gene_name_mouse
0  ENSG00000198888          MT-ND1  ENSMUSG00000064341          mt-Nd1
1  ENSG00000198763          MT-ND2  ENSMUSG00000064345          mt-Nd2
2  ENSG00000198804          MT-CO1  ENSMUSG00000064351          mt-Co1
3  ENSG00000198712          MT-CO2  ENSMUSG00000064354          mt-Co2
4  ENSG00000228253         MT-ATP8  ENSMUSG00000064356         mt-Atp8

In [29]:
orth_genes["orth_id"] = orth_genes["gene_id_human"] + "__" + orth_genes["gene_id_mouse"]
orth_genes.head()

gene_id_human gene_name_human       gene_id_mouse gene_name_mouse  \
0  ENSG00000198888          MT-ND1  ENSMUSG00000064341          mt-Nd1   
1  ENSG00000198763          MT-ND2  ENSMUSG00000064345          mt-Nd2   
2  ENSG00000198804          MT-CO1  ENSMUSG00000064351          mt-Co1   
3  ENSG00000198712          MT-CO2  ENSMUSG00000064354          mt-Co2   
4  ENSG00000228253         MT-ATP8  ENSMUSG00000064356         mt-Atp8   

                               orth_id  
0  ENSG00000198888__ENSMUSG00000064341  
1  ENSG00000198763__ENSMUSG00000064345  
2  ENSG00000198804__ENSMUSG00000064351  
3  ENSG00000198712__ENSMUSG00000064354  
4  ENSG00000228253__ENSMUSG00000064356

In [30]:
hESC_orth = hESC.merge(orth_genes, left_on="gene_id", right_on="gene_id_human")
hESC_orth.head()

gene_id  rep1  rep2    gene_id_human gene_name_human  \
0  ENSG00000187634    91    67  ENSG00000187634          SAMD11   
1  ENSG00000188976  7273  4400  ENSG00000188976           NOC2L   
2  ENSG00000187961   975   697  ENSG00000187961          KLHL17   
3  ENSG00000187642    11     4  ENSG00000187642           PERM1   
4  ENSG00000187608   546   484  ENSG00000187608           ISG15   

        gene_id_mouse gene_name_mouse                              orth_id  
0  ENSMUSG00000096351          Samd11  ENSG00000187634__ENSMUSG00000096351  
1  ENSMUSG00000095567           Noc2l  ENSG00000188976__ENSMUSG00000095567  
2  ENSMUSG00000078485         Plekhn1  ENSG00000187961__ENSMUSG00000078485  
3  ENSMUSG00000078486           Perm1  ENSG00000187642__ENSMUSG00000078486  
4  ENSMUSG00000035692           Isg15  ENSG00000187608__ENSMUSG00000035692

In [31]:
hESC_orth = hESC_orth[["orth_id", "rep1", "rep2"]].drop_duplicates()
print(len(hESC_orth))
hESC_orth.head()

19978


orth_id  rep1  rep2
0  ENSG00000187634__ENSMUSG00000096351    91    67
1  ENSG00000188976__ENSMUSG00000095567  7273  4400
2  ENSG00000187961__ENSMUSG00000078485   975   697
3  ENSG00000187642__ENSMUSG00000078486    11     4
4  ENSG00000187608__ENSMUSG00000035692   546   484

In [32]:
mESC_orth = mESC.merge(orth_genes, left_on="gene_id", right_on="gene_id_mouse")
mESC_orth.head()

gene_id  rep1  rep2  rep3    gene_id_human gene_name_human  \
0  ENSMUSG00000064842     0     0     0  ENSG00000252998       RNU6-889P   
1  ENSMUSG00000051951     2     2     2  ENSG00000206579            XKR4   
2  ENSMUSG00000025900     0     0     0  ENSG00000104237             RP1   
3  ENSMUSG00000025902    14    18    26  ENSG00000164736           SOX17   
4  ENSMUSG00000096126     0     0     0  ENSG00000200887       RNU6-598P   

        gene_id_mouse gene_name_mouse                              orth_id  
0  ENSMUSG00000064842         Gm26206  ENSG00000252998__ENSMUSG00000064842  
1  ENSMUSG00000051951            Xkr4  ENSG00000206579__ENSMUSG00000051951  
2  ENSMUSG00000025900             Rp1  ENSG00000104237__ENSMUSG00000025900  
3  ENSMUSG00000025902           Sox17  ENSG00000164736__ENSMUSG00000025902  
4  ENSMUSG00000096126         Gm22307  ENSG00000200887__ENSMUSG00000096126

In [33]:
mESC_orth = mESC_orth[["orth_id", "rep1", "rep2", "rep3"]].drop_duplicates()
print(len(mESC_orth))
mESC_orth.head()

20379


orth_id  rep1  rep2  rep3
0  ENSG00000252998__ENSMUSG00000064842     0     0     0
1  ENSG00000206579__ENSMUSG00000051951     2     2     2
2  ENSG00000104237__ENSMUSG00000025900     0     0     0
3  ENSG00000164736__ENSMUSG00000025902    14    18    26
4  ENSG00000200887__ENSMUSG00000096126     0     0     0

In [34]:
orth_counts = hESC_orth.merge(mESC_orth, on="orth_id", suffixes=("_x", "_y"))
print(len(orth_counts))
print(len(orth_counts.orth_id.unique()))
orth_counts.columns = ["orth_id", "hESC_rep1", "hESC_rep2", "mESC_rep1", "mESC_rep2", "mESC_rep3"]
orth_counts.head()

19858
19858


orth_id  hESC_rep1  hESC_rep2  mESC_rep1  \
0  ENSG00000187634__ENSMUSG00000096351         91         67          5   
1  ENSG00000188976__ENSMUSG00000095567       7273       4400      13774   
2  ENSG00000187961__ENSMUSG00000078485        975        697        657   
3  ENSG00000187642__ENSMUSG00000078486         11          4         28   
4  ENSG00000187608__ENSMUSG00000035692        546        484          6   

   mESC_rep2  mESC_rep3  
0          5         10  
1      14229      18741  
2        751        929  
3         22         20  
4          5          5

In [35]:
orth_cols = {"hESC_rep1": ["hESC", "sample1"], "hESC_rep2": ["hESC", "sample2"], 
             "mESC_rep1": ["mESC", "sample1"], "mESC_rep2": ["mESC", "sample2"], "mESC_rep3": ["mESC", "sample3"]}
orth_cols = pd.DataFrame.from_dict(orth_cols, orient="index").reset_index()
orth_cols.columns = ["column", "condition", "sample"]
orth_cols.head()

column condition   sample
0  hESC_rep1      hESC  sample1
1  hESC_rep2      hESC  sample2
2  mESC_rep1      mESC  sample1
3  mESC_rep2      mESC  sample2
4  mESC_rep3      mESC  sample3

## 6. write files

In [36]:
hESC.to_csv("%s/hESC_all.counts.txt" % rna_seq_dir, sep="\t", index=False)
mESC.to_csv("%s/mESC_all.counts.txt" % rna_seq_dir, sep="\t", index=False)
hESC_cols.to_csv("%s/hESC_all.cols.txt" % rna_seq_dir, sep="\t", index=False)
mESC_cols.to_csv("%s/mESC_all.cols.txt" % rna_seq_dir, sep="\t", index=False)

In [37]:
orth_counts.to_csv("%s/orths.counts.txt" % rna_seq_dir, sep="\t", index=False)
orth_cols.to_csv("%s/orths.cols.txt" % rna_seq_dir, sep="\t", index=False)